In [1]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_hub as hub

df = pd.read_csv("nofo_final_sub.csv", header=None)

In [3]:
# index = 0
# for i in df[0]:
#     if type(i) != str:
#         df = df.drop([index])
#     index += 1

# for i in df[0]:
#     if type(i) != str:
#         print(type(i))

In [4]:
# df.head()

In [5]:
# def bold_label(row):
#     if "Bold" in row[4]:
#         return 1
#     return 0

# df[5] = df.apply(lambda row: bold_label(row), axis=1)

In [6]:
# def agency_label(row):
#     if "USAID" in row[5]:
#         return 1
#     return 0

# df[7] = df.apply(lambda row: agency_label(row), axis=1)

In [7]:
df = df[df[2] != 1] #remove section headers

In [8]:
train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9*len(df))]) #60 training, 20 validation, 20 test
#can go to 80/10/10 for larger sets, or even more...

len(train), len(val), len(test)

(34032, 4254, 4255)

In [9]:
def df_to_dataset(dataframe, shuffle=True, batch_size=256):
  df = dataframe.copy()
  labels = df.pop(1)
  df = df[0]
  ds = tf.data.Dataset.from_tensor_slices((df, labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

train_data = df_to_dataset(train)
valid_data = df_to_dataset(val)
test_data = df_to_dataset(test)

In [10]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, dtype=tf.string, trainable=True)

In [11]:
model = tf.keras.Sequential()
model.add(hub_layer) #first layer in neural net is text to value
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [13]:
model.evaluate(train_data)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


133/133 [==============================] - 1s 4ms/step - loss: 0.6641 - accuracy: 0.7782


[0.66411954164505, 0.7782087326049805]

In [14]:
model.evaluate(valid_data)

17/17 [==============================] - 0s 4ms/step - loss: 0.6633 - accuracy: 0.7920


[0.6632631421089172, 0.7919605374336243]

In [15]:
history = model.fit(train_data, epochs=10, validation_data=valid_data)

Epoch 1/10
133/133 [==============================] - 92s 684ms/step - loss: 0.2985 - accuracy: 0.9272 - val_loss: 0.1281 - val_accuracy: 0.9509
Epoch 2/10
133/133 [==============================] - 91s 682ms/step - loss: 0.1244 - accuracy: 0.9520 - val_loss: 0.0991 - val_accuracy: 0.9596
Epoch 3/10
133/133 [==============================] - 95s 717ms/step - loss: 0.0877 - accuracy: 0.9632 - val_loss: 0.0928 - val_accuracy: 0.9687
Epoch 4/10
133/133 [==============================] - 93s 698ms/step - loss: 0.0682 - accuracy: 0.9729 - val_loss: 0.0966 - val_accuracy: 0.9701
Epoch 5/10
133/133 [==============================] - 91s 681ms/step - loss: 0.0539 - accuracy: 0.9808 - val_loss: 0.1037 - val_accuracy: 0.9704
Epoch 6/10
133/133 [==============================] - 95s 712ms/step - loss: 0.0447 - accuracy: 0.9843 - val_loss: 0.1097 - val_accuracy: 0.9697
Epoch 7/10
133/133 [==============================] - 88s 659ms/step - loss: 0.0389 - accuracy: 0.9859 - val_loss: 0.1242 - val_ac

In [16]:
model.evaluate(test_data)

17/17 [==============================] - 2s 104ms/step - loss: 0.1303 - accuracy: 0.9746


[0.13031244277954102, 0.9746180772781372]

In [17]:
examples = df[0].values.tolist()
print(examples)
test_predict = model.predict(examples)

["b'Issue Date:  '", "b'November 10, 2022  '", "b'Deadline for Questions:                       '", "b'November 30, 2022  '", "b'Closing Date:  '", "b'January 11, 2023 '", "b'Closing Time:  '", "b'09:00 am, Phnom Penh Time '", "b'Subject:   '", "b'Notice of Funding Opportunity Number: 72044223RFA00001 '", "b'Program Title:      USAID Enhancing Quality of Healthcare Activity II (EQHA II) '", "b'Ladies/Gentlemen: '", "b'The United States Agency for International Development (USAID) is seeking applications for a '", "b'cooperative agreement from qualified entities excluding all countries not included in Geographic '", "b'Code 937, to implement a program entitled \\xe2\\x80\\x9cEnhancing Quality of Healthcare Activity II\\xe2\\x80\\x9d.  '", "b'Applicants under consideration for an award that have never received funding from USAID may be '", "b'subject to a pre-award audit to determine fiscal responsibility, ensure adequacy of financial controls '", "b'and establish an indirect cost rate. 

In [18]:
outcome = []
for i in test_predict:
    outcome.append(i[0])
df['text_predictions'] = outcome

In [19]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,text_predictions
0,b'Issue Date: ',0,NaN,12.0,TimesNewRomanPSMT,0,1,72.0,131.639999,100.126587,113.415649,612,792,nofo.pdf,4.515986e-01
1,"b'November 10, 2022 '",0,NaN,12.0,TimesNewRomanPSMT,0,1,288.0,399.000000,100.126587,113.415649,612,792,nofo.pdf,4.447944e-07
2,b'Deadline for Questions: ',0,NaN,12.0,TimesNewRomanPSMT,0,1,72.0,255.660004,113.926575,127.215637,612,792,nofo.pdf,7.548132e-01
3,"b'November 30, 2022 '",0,NaN,12.0,TimesNewRomanPSMT,0,1,288.0,399.000000,113.926575,127.215637,612,792,nofo.pdf,7.184435e-10
4,b'Closing Date: ',0,NaN,12.0,TimesNewRomanPSMT,0,1,72.0,147.000000,127.726562,141.015625,612,792,nofo.pdf,3.588808e-01


In [20]:
df = df[[3, 5, 7, 8, 9, 10, 'text_predictions', 1]]

In [21]:
spec = []
for i in df[5]:
    if i != 0:
        spec.append(1)
    else:
        spec.append(0)

In [22]:
df[5] = spec

In [23]:
# for i in range(len(df.columns[:-1])):
#     label = df.columns[i]
#     plt.hist(df[df[1]==1][label],color='blue', label='Subheader',alpha=0.7)
#     plt.hist(df[df[1]==0][label],color='red', label='Not Header',alpha=0.7)
#     plt.title(label)
#     plt.ylabel('N')
#     plt.xlabel(label)
#     plt.legend()
#     plt.show()

In [24]:
x = df[df.columns[:-1]].values
y = df[df.columns[-1]].values

scaler = StandardScaler()
x = scaler.fit_transform(x)

In [25]:
# over = RandomOverSampler()
# x, y = over.fit_resample(x,y)

x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.4, random_state=0)
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=0)

In [26]:
model_2 = tf.keras.Sequential()
# model_2.add(hub_layer) #first layer in neural net is text to value
model_2.add(tf.keras.layers.Dense(16, activation='relu'))
model_2.add(tf.keras.layers.Dropout(0.4))
model_2.add(tf.keras.layers.Dense(16, activation='relu'))
model_2.add(tf.keras.layers.Dropout(0.4))
model_2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [27]:
model_2.evaluate(x_train, y_train)

798/798 [==============================] - 3s 3ms/step - loss: 0.7090 - accuracy: 0.4696


[0.7089766263961792, 0.46955806016921997]

In [28]:
model_2.fit(x_train, y_train, batch_size=16, epochs=20, validation_data=(x_valid, y_valid))

Epoch 1/20
1596/1596 [==============================] - 9s 4ms/step - loss: 0.1141 - accuracy: 0.9649 - val_loss: 0.0277 - val_accuracy: 0.9915
Epoch 2/20
1596/1596 [==============================] - 5s 3ms/step - loss: 0.0499 - accuracy: 0.9840 - val_loss: 0.0276 - val_accuracy: 0.9913
Epoch 3/20
1596/1596 [==============================] - 7s 4ms/step - loss: 0.0416 - accuracy: 0.9875 - val_loss: 0.0280 - val_accuracy: 0.9919
Epoch 4/20
1596/1596 [==============================] - 6s 4ms/step - loss: 0.0412 - accuracy: 0.9882 - val_loss: 0.0268 - val_accuracy: 0.9915
Epoch 5/20
1596/1596 [==============================] - 7s 4ms/step - loss: 0.0387 - accuracy: 0.9891 - val_loss: 0.0269 - val_accuracy: 0.9917
Epoch 6/20
1596/1596 [==============================] - 6s 4ms/step - loss: 0.0379 - accuracy: 0.9896 - val_loss: 0.0275 - val_accuracy: 0.9915
Epoch 7/20
1596/1596 [==============================] - 5s 3ms/step - loss: 0.0361 - accuracy: 0.9895 - val_loss: 0.0268 - val_accuracy:

In [29]:
model_2.evaluate(x_test, y_test)

266/266 [==============================] - 0s 1ms/step - loss: 0.0291 - accuracy: 0.9900


[0.029105169698596, 0.9900105595588684]

In [30]:
# model.save('subheader_models/model_1')
# model_2.save('subheader_models/model_2')

In [31]:
df_2 = pd.read_csv("test.csv", header=None)

In [32]:
# df_2.head()

In [33]:
examples = df_2[0].values.tolist()
# print(examples)

In [34]:
test_predict = model.predict(examples)

27/27 [==============================] - 2s 61ms/step


In [35]:
# # print(test_predict)
outcome = []
for i in test_predict:
    outcome.append(i[0]*100)

In [36]:
df_2['outcome'] = outcome

In [37]:
df_2.to_csv("out.csv")

In [38]:
# encoder = tf.keras.layers.TextVectorization(max_tokens=2000)
# encoder.adapt(train_data.map(lambda text, label: text))

In [39]:
# vocab = np.array(encoder.get_vocabulary())
# vocab[:20]

In [40]:
# model = tf.keras.Sequential([
#     encoder,
#     tf.keras.layers.Embedding(
#         input_dim=len(encoder.get_vocabulary()),
#         output_dim=32,
#         mask_zero=True
#     ),
#     tf.keras.layers.LSTM(32),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

In [41]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss=tf.keras.losses.BinaryCrossentropy(),
#               metrics=['accuracy'])

In [42]:
# model.evaluate(train_data)
# model.evaluate(valid_data)

In [43]:
# history = model.fit(train_data, epochs=10, validation_data=valid_data)

In [44]:
# model.evaluate(test_data)